In [13]:
import time, json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup, SoupStrainer
from bs4.diagnose import diagnose
from parsel import Selector
from selenium.webdriver.common.action_chains import ActionChains

driver = webdriver.Chrome()
ecology = driver.get("https://esajournals.onlinelibrary.wiley.com/loi/19399170")
ecology_content = driver.page_source
issue_archive = SoupStrainer(class_='rlist loi__list')
browse_soup = BeautifulSoup(ecology_content, 'html.parser', parse_only = issue_archive)

volume_links = []

for vol_link in browse_soup.find_all('a'):
    volume_links.append(vol_link.get('href'))
    
##After getting all the volume links we acces them

issue_links = []
for volume in volume_links:
    eco_volume = driver.get("https://esajournals.onlinelibrary.wiley.com" + volume)
    eco_volume_content = driver.page_source
    volume_issues = SoupStrainer(class_='rlist loi__issues')
    browse_volume_issues = BeautifulSoup(eco_volume_content, 'html.parser', parse_only = volume_issues)
    
    for issue_link in browse_volume_issues.find_all('a'):
        issue_links.append(issue_link.get('href'))

##After getting all the issue links in the volume, access and extract titles and related links
titles = []
doi = []
for issue in issue_links:
    volume_issue = driver.get("https://esajournals.onlinelibrary.wiley.com" + issue)
    volume_issue_content = driver.page_source
    articles = SoupStrainer(class_='issue-item__title visitable')
    browse_articles = BeautifulSoup(volume_issue_content, 'html.parser', parse_only = articles)
    
    for title in browse_articles.find_all('h2'):
        titles.append(title.get_text())
        
    for title_doi in browse_articles.find_all('a'):
        doi.append(title_doi.get('href'))


article_dict = dict(zip(titles, doi))
with open('Ecology.json', 'w+') as fp:
    json.dump(article_dict, fp,indent = "\n")